# Preprocessing: BJH050


This markdown files loads the clean data and does a bipolar rereference as well as epochs the data


In [1]:
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import re
import os
import mne
import IPython
import seaborn as sns

In [2]:
## Prep paths ##

subject = 'BJH050'
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}"
preproc_data_dir = f"/home/brooke/pacman/preprocessing/{subject}/ieeg"

In [3]:
## Load Data ##

# load filtered data #
filtered_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_notched_filtered_clean_ieeg.fif")

# load raw data #
raw_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_raw_clean_ieeg.fif")



Opening raw data file /home/brooke/pacman/raw_data/BJH050/ieeg/BJH050_notched_filtered_clean_ieeg.fif...
    Range : 0 ... 2085999 =      0.000 ...  1042.999 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/BJH050/ieeg/BJH050_notched_filtered_clean_ieeg-1.fif...
    Range : 2086000 ... 2992399 =   1043.000 ...  1496.199 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/BJH050/ieeg/BJH050_raw_clean_ieeg.fif...
    Range : 0 ... 2085999 =      0.000 ...  1042.999 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/BJH050/ieeg/BJH050_raw_clean_ieeg-1.fif...
    Range : 2086000 ... 2992399 =   1043.000 ...  1496.199 secs
Ready.


## Bipolar Rereferencing

In [7]:
# helper functions

def probe_and_num(elec_str): 
    ''' This convenience function takes an electrode string like LHH1 and outputs ('LHH', 1)
    '''
    if '_' in elec_str:
        return None, None
    else:   
        regex_str = '(\D+)(\d+)' # group of letters followed by group of digits
        matcher = re.compile(regex_str,re.IGNORECASE|re.DOTALL).search(elec_str)
        if matcher:
            probe, num = matcher.groups()
            return probe, int(num)
        else:
            return None, None
    
def find_bipolar_pair(ch, labels, remove):
    ''' Find the pair of a given electrode for bipolar referencing.
        Given a single *ch* and a list of *labels*, some of which you want to *remove*,
        this finds the next channel on that probe that isn't meant to be removed (WM is ok).
    '''
    bipolar_pair = None
    probe, num = probe_and_num(ch)
    other_in_probe = [other_ch for other_ch in labels if probe_and_num(other_ch)[0]==probe]
    for i in range(len(other_in_probe)-num):
        next_ch = probe + str(num+i+1)
        if next_ch in remove or next_ch == 'STI':
            continue
        else:
            bipolar_pair = next_ch
            break
    return bipolar_pair

In [5]:
mne.channels.rename_channels(filtered_clean_fif.info, {' AR13': 'AR13'})
mne.channels.rename_channels(raw_clean_fif.info, {' AR13': 'AR13'})

In [8]:
# Here I iterate through the electrodes in my ROIs and match them up with their bipolar pair
pairs = []
anode = []
cathode = []
pairs_name = []
pairs_map = {}
remove = filtered_clean_fif.info['bads']
labels = filtered_clean_fif.info['ch_names']
for ch in labels:
    # if a channel is meant to be removed, it doesn't get to be in a bipolar pair
    if ch in remove or ch == 'STI':
        print(f"{ch} noref")
    else:
        pair = find_bipolar_pair(ch, labels, remove)
        if pair:
            anode.append(ch)
            cathode.append(pair)
            pairs.append((ch, pair))
            pairs_map[ch] = f"{ch}-{pair}"
            pairs_name.append(f"{ch}-{pair}")
            print(ch, pair)

AR1 AR6
AR2 noref
AR3 noref
AR4 noref
REF1 noref
REF2 noref
AR5 noref
AR6 AR7
AR7 AR8
AR8 AR9
AR9 AR10
AR10 AR11
AR11 AR12
AR12 AR13
AR13 AR14
BR1 noref
BR2 noref
BR3 BR4
BR4 BR5
BR5 BR6
BR6 BR7
BR7 BR8
CR1 noref
CR2 noref
CR3 CR4
CR4 CR5
CR5 CR6
CR6 CR7
CR7 CR8
CR8 CR9
CR9 CR10
CR10 CR11
CR11 CR12
DR1 DR2
DR2 DR3
DR3 DR4
DR4 DR5
DR5 DR6
DR6 DR7
DR7 DR8
ER1 noref
ER2 noref
ER3 noref
ER4 noref
ER5 ER6
ER6 ER7
ER7 ER8
ER8 ER9
ER9 ER10
FR1 noref
FR2 noref
FR3 noref
FR4 FR5
FR5 FR6
FR6 FR7
FR7 FR8
FR8 FR9
FR9 FR10
FR10 FR11
FR11 FR12
GR1 GR2
GR2 GR3
GR3 GR4
GR4 GR5
GR5 GR6
GR6 GR7
GR8 noref
HR1 noref
HR2 noref
HR3 noref
HR4 noref
HR5 noref
HR6 noref
HR7 noref
HR8 noref
HR9 HR10
IR1 IR2
IR2 IR3
IR3 IR4
IR4 IR5
IR5 IR9
IR6 noref
IR7 noref
IR8 noref
IR9 IR10
JR1 JR2
JR2 JR3
JR3 JR4
JR4 JR5
JR5 JR6
JR6 JR7
JR7 JR8
KR1 KR2
KR2 KR3
KR3 KR4
KR4 KR5
KR5 KR6
KR6 KR7
KR7 KR8
KR8 KR9
KR9 KR10
KR10 KR11
KR11 KR12
KR12 KR13
KR13 KR14
LR1 LR2
LR2 LR3
LR3 LR4
LR4 LR5
LR5 LR6
LR6 LR7
LR7 LR8
LR8 LR9
LR9 L

In [9]:
## Apply Rereference #

if filtered_clean_fif.info['ch_names'] == raw_clean_fif.info['ch_names'] and filtered_clean_fif.info['bads'] == raw_clean_fif.info['bads']:
    
    # load filtered data
    filtered_clean_fif.load_data()
    
    # set filtered reference 
    bp_filt_fif = mne.set_bipolar_reference(filtered_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)
    
    # clear var
    bp_filt_fif = []
    filtered_clean_fif = []
    
    # load raw data
    raw_clean_fif.load_data()
    
    # set raw reference 
    bp_raw_fif = mne.set_bipolar_reference(raw_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)

    
    # clear var
    bp_raw_fif = []
    raw_clean_fif = []    
    
    

Reading 0 ... 2992399  =      0.000 ...  1496.199 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=127, n_times=2992400
    Range : 0 ... 2992399 =      0.000 ...  1496.199 secs
Ready.
Added the following bipolar channels:
AR1-AR6, AR6-AR7, AR7-AR8, AR8-AR9, AR9-AR10, AR10-AR11, AR11-AR12, AR12-AR13, AR13-AR14, BR3-BR4, BR4-BR5, BR5-BR6, BR6-BR7, BR7-BR8, CR3-CR4, CR4-CR5, CR5-CR6, CR6-CR7, CR7-CR8, CR8-CR9, CR9-CR10, CR10-CR11, CR11-CR12, DR1-DR2, DR2-DR3, DR3-DR4, DR4-DR5, DR5-DR6, DR6-DR7, DR7-DR8, ER5-ER6, ER6-ER7, ER7-ER8, ER8-ER9, ER9-ER10, FR4-FR5, FR5-FR6, FR6-FR7, FR7-FR8, FR8-FR9, FR9-FR10, FR10-FR11, FR11-FR12, GR1-GR2, GR2-GR3, GR3-GR4, GR4-GR5, GR5-GR6, GR6-GR7, HR9-HR10, IR1-IR2, IR2-IR3, IR3-IR4, IR4-IR5, IR5-IR9, IR9-IR10, JR1-JR2, JR2-JR3, JR3-JR4, JR4-JR5, JR5-JR6, JR6-JR7, JR7-JR8, KR1-KR2, KR2-KR3, KR3-KR4, KR4-KR5, KR5-KR6, KR6-KR7, KR7-KR8, KR8-KR9, KR9-KR10, KR10-KR11, KR11-KR12, KR12-KR13, KR13-KR14, LR1-LR2, 

/tmp/ipykernel_8399/1336304392.py:12: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH050/ieeg/BJH050_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)


Closing /home/brooke/pacman/preprocessing/BJH050/ieeg/BJH050_bp_filtered_clean_data.fif
Overwriting existing file.
Writing /home/brooke/pacman/preprocessing/BJH050/ieeg/BJH050_bp_filtered_clean_data-1.fif
Closing /home/brooke/pacman/preprocessing/BJH050/ieeg/BJH050_bp_filtered_clean_data-1.fif
[done]
Reading 0 ... 2992399  =      0.000 ...  1496.199 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=127, n_times=2992400
    Range : 0 ... 2992399 =      0.000 ...  1496.199 secs
Ready.
Added the following bipolar channels:
AR1-AR6, AR6-AR7, AR7-AR8, AR8-AR9, AR9-AR10, AR10-AR11, AR11-AR12, AR12-AR13, AR13-AR14, BR3-BR4, BR4-BR5, BR5-BR6, BR6-BR7, BR7-BR8, CR3-CR4, CR4-CR5, CR5-CR6, CR6-CR7, CR7-CR8, CR8-CR9, CR9-CR10, CR10-CR11, CR11-CR12, DR1-DR2, DR2-DR3, DR3-DR4, DR4-DR5, DR5-DR6, DR6-DR7, DR7-DR8, ER5-ER6, ER6-ER7, ER7-ER8, ER8-ER9, ER9-ER10, FR4-FR5, FR5-FR6, FR6-FR7, FR7-FR8, FR8-FR9, FR9-FR10, FR10-FR11, FR11-FR12, GR1-GR2, GR2-GR

/tmp/ipykernel_8399/1336304392.py:25: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH050/ieeg/BJH050_bp_raw_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)


Closing /home/brooke/pacman/preprocessing/BJH050/ieeg/BJH050_bp_raw_clean_data.fif
Overwriting existing file.
Writing /home/brooke/pacman/preprocessing/BJH050/ieeg/BJH050_bp_raw_clean_data-1.fif
Closing /home/brooke/pacman/preprocessing/BJH050/ieeg/BJH050_bp_raw_clean_data-1.fif
[done]


In [ ]:
# Visualize it #

# bp_raw_fif.plot(events=events, color='b', bad_color = 'cyan', n_channels = 1, clipping = None, event_color = 'r')

## Epoching the data 

### Onset

In [10]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 1, tmin = -4, tmax = 12, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

Opening raw data file /home/brooke/pacman/preprocessing/BJH050/ieeg/BJH050_bp_filtered_clean_data.fif...
    Range : 0 ... 2215999 =      0.000 ...  1107.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH050/ieeg/BJH050_bp_filtered_clean_data-1.fif...
    Range : 2216000 ... 2992399 =   1108.000 ...  1496.199 secs
Ready.


/tmp/ipykernel_8399/1463149468.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH050/ieeg/BJH050_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


480 events found on stim channel STI
Event IDs: [0 1]
Not setting metadata
240 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 32001 original time points ...
Splitting into 2 parts
Overwriting existing file.


/tmp/ipykernel_8399/1463149468.py:18: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH050/ieeg/BJH050_bp_clean_pres-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


Loading data for 120 events and 32001 original time points ...
Overwriting existing file.
Loading data for 120 events and 32001 original time points ...


### Trial End

In [11]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
events = events[0:-1] # remove last event
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 0, tmin = -5, tmax = 5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

Opening raw data file /home/brooke/pacman/preprocessing/BJH050/ieeg/BJH050_bp_filtered_clean_data.fif...
    Range : 0 ... 2215999 =      0.000 ...  1107.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH050/ieeg/BJH050_bp_filtered_clean_data-1.fif...
    Range : 2216000 ... 2992399 =   1108.000 ...  1496.199 secs
Ready.


/tmp/ipykernel_8399/3432568909.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH050/ieeg/BJH050_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


480 events found on stim channel STI
Event IDs: [0 1]
Not setting metadata
239 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 20001 original time points ...
Splitting into 2 parts
Overwriting existing file.
Loading data for 120 events and 20001 original time points ...


/tmp/ipykernel_8399/3432568909.py:19: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH050/ieeg/BJH050_bp_clean_end-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


Overwriting existing file.
Loading data for 119 events and 20001 original time points ...


### Last Away

In [12]:
## Load Behavioral Data ##
last_away_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_last_away_events.csv")
last_away_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,16,40200,0,1
1,1,9,45800,0,1
2,2,13,61700,0,1
3,3,10,73200,0,1
4,5,15,100900,0,1
...,...,...,...,...,...
204,232,3,2764600,0,1
205,233,13,2775300,0,1
206,236,7,2818800,0,1
207,237,16,2830400,0,1


In [13]:
# create events
last_away_events = last_away_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [14]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
last_away_epochs = mne.Epochs(bp_filtered_fif, last_away_events, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH050/ieeg/BJH050_bp_filtered_clean_data.fif...
    Range : 0 ... 2215999 =      0.000 ...  1107.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH050/ieeg/BJH050_bp_filtered_clean_data-1.fif...
    Range : 2216000 ... 2992399 =   1108.000 ...  1496.199 secs
Ready.
Not setting metadata
209 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 20001 original time points ...
Splitting into 2 parts
Overwriting existing file.


/tmp/ipykernel_8399/678285560.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH050/ieeg/BJH050_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_8399/678285560.py:13: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH050/ieeg/BJH050_bp_filtered_clean_last_away_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


Loading data for 105 events and 20001 original time points ...
Overwriting existing file.
Loading data for 104 events and 20001 original time points ...


### First Dot

In [15]:
## Load Behavioral Data ##
first_dot_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_dot_events.csv")
first_dot_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,2,13,52500,0,1
1,3,10,70900,0,1
2,4,6,82900,0,1
3,5,15,96400,0,1
4,6,5,110700,0,1
...,...,...,...,...,...
220,234,6,2784700,0,1
221,235,5,2799200,0,1
222,236,7,2817200,0,1
223,237,16,2827500,0,1


In [16]:
# create events
first_dot_data = first_dot_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [17]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_dot_epochs = mne.Epochs(bp_filtered_fif, first_dot_data, 
                          event_id = 1, tmin = -3, tmax =6, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH050/ieeg/BJH050_bp_filtered_clean_data.fif...
    Range : 0 ... 2215999 =      0.000 ...  1107.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH050/ieeg/BJH050_bp_filtered_clean_data-1.fif...
    Range : 2216000 ... 2992399 =   1108.000 ...  1496.199 secs
Ready.
Not setting metadata
225 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 18001 original time points ...
Splitting into 2 parts
Overwriting existing file.
Loading data for 113 events and 18001 original time points ...


/tmp/ipykernel_8399/4126832509.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH050/ieeg/BJH050_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_8399/4126832509.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH050/ieeg/BJH050_bp_filtered_clean_first_dot_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif", overwrite = True, split_size = '1.9GB')


Overwriting existing file.
Loading data for 112 events and 18001 original time points ...


## First Move

In [18]:
## Load Behavioral Data ##
first_move_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_move_events.csv")
first_move_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,16,40200,0,1
1,1,9,45800,0,1
2,2,13,51600,0,1
3,3,10,68600,0,1
4,4,6,82500,0,1
...,...,...,...,...,...
234,234,6,2784500,0,1
235,235,5,2798500,0,1
236,236,7,2816600,0,1
237,237,16,2827300,0,1


In [19]:
# create events
first_move_data = first_move_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [20]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_move_epochs = mne.Epochs(bp_filtered_fif, first_move_data, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_move_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_move_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH050/ieeg/BJH050_bp_filtered_clean_data.fif...
    Range : 0 ... 2215999 =      0.000 ...  1107.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH050/ieeg/BJH050_bp_filtered_clean_data-1.fif...
    Range : 2216000 ... 2992399 =   1108.000 ...  1496.199 secs
Ready.
Not setting metadata
239 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 20001 original time points ...
Splitting into 2 parts
Overwriting existing file.
Loading data for 120 events and 20001 original time points ...


/tmp/ipykernel_8399/3347570552.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH050/ieeg/BJH050_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_8399/3347570552.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH050/ieeg/BJH050_bp_filtered_clean_first_move_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_move_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_move_events.fif", overwrite = True, split_size = '1.9GB')


Overwriting existing file.
Loading data for 119 events and 20001 original time points ...


## Ghost Attack

In [21]:
## Load Behavioral Data ##
ghost_attack_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_attack_events.csv")
ghost_attack_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,2,13,60800,0,1
1,4,6,84800,0,1
2,7,7,123800,0,1
3,8,2,143000,0,1
4,15,14,229400,0,1
5,16,12,240700,0,1
6,25,1,363800,0,1
7,31,12,431900,0,1
8,41,10,548500,0,1
9,44,11,582800,0,1


In [22]:
# create events
ghost_attack_data = ghost_attack_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [23]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
ghost_attack_epochs = mne.Epochs(bp_filtered_fif, ghost_attack_data, 
                          event_id = 1, tmin = -4, tmax =6, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
ghost_attack_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_ghost_attack_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH050/ieeg/BJH050_bp_filtered_clean_data.fif...
    Range : 0 ... 2215999 =      0.000 ...  1107.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH050/ieeg/BJH050_bp_filtered_clean_data-1.fif...
    Range : 2216000 ... 2992399 =   1108.000 ...  1496.199 secs
Ready.
Not setting metadata
53 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 20001 original time points ...
Overwriting existing file.
Loading data for 53 events and 20001 original time points ...


/tmp/ipykernel_8399/2681655913.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH050/ieeg/BJH050_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_8399/2681655913.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH050/ieeg/BJH050_bp_filtered_clean_ghost_attack_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  ghost_attack_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_ghost_attack_events.fif", overwrite = True, split_size = '1.9GB')
